# 配列のsortとargsort
## Agenda
- `numpy.sort`と`numpy.argsort`の紹介

### Import

In [1]:
import numpy as np

### Numpy version

In [2]:
np.__version__

'1.18.2'

## 1. `numpy.sort`

- sorted copy of an arrayを返す

### Syntax

```
numpy.sort(a, axis=-1, kind=None, order=None)[source]
```

### Parameters

|引数名|型|概要|
|---|---|---|
|`a`|array_like|sortしたい配列を指定|
|`axis`|int or None|default `axis = -1`, sortするaxisの指定|
|`kind`|`{‘quicksort’, ‘mergesort’, ‘heapsort’, ‘stable’}, optional`|defaultはquicksort|
|`order`|string, list of strings|aがフィールドの定義されている配列であったとき、どのフィールドで要素をソートするかを指定する|


### アルゴリズムの性能評価の指標

- average speed
- worst case performance
- workspace size
- stability

<img src = 'https://github.com/RyoNakagami/omorikaizuka/blob/master/algorithm/sort_algorithm.jpg?raw=true'>

In [3]:
np.random.seed(42)
a = np.random.randint(0, 100, size = 10)
a

array([51, 92, 14, 71, 60, 20, 82, 86, 74, 74])

In [4]:
np.sort(a)

array([14, 20, 51, 60, 71, 74, 74, 82, 86, 92])

In [5]:
a = np.array([[1,4],[3,1], [5,0]])
np.sort(a)

array([[1, 4],
       [1, 3],
       [0, 5]])

In [6]:
np.sort(a, axis=None)

array([0, 1, 1, 3, 4, 5])

In [7]:
a = np.array([[1,4],[3,1], [5,0]])
np.sort(a, axis=0)

array([[1, 0],
       [3, 1],
       [5, 4]])

### order

In [8]:
dtype = [('name',  np.unicode_, 16), ('height', float), ('age', int)]
values = [('Arthur', 1.8, 41), ('Lancelot', 1.9, 38),
           ('Galahad', 1.7, 38)]
a = np.array(values, dtype=dtype)  
np.sort(a, order = 'height')

array([('Galahad', 1.7, 38), ('Arthur', 1.8, 41), ('Lancelot', 1.9, 38)],
      dtype=[('name', '<U16'), ('height', '<f8'), ('age', '<i8')])

In [9]:
np.sort(a, order=['age', 'height'])       

array([('Galahad', 1.7, 38), ('Lancelot', 1.9, 38), ('Arthur', 1.8, 41)],
      dtype=[('name', '<U16'), ('height', '<f8'), ('age', '<i8')])

## 2. `nunpy.argsort`

- sort結果の配列indexを返す

### Syntax

```
numpy.argsort(a, axis=-1, kind=None, order=None)
```

In [10]:
x = np.array([3, 1, 2])
np.argsort(x)

array([1, 2, 0])

In [11]:
a = np.array([[1,4],[3,1], [5,2]])
np.argsort(a, axis=1)

array([[0, 1],
       [1, 0],
       [1, 0]])

In [12]:
a = np.array([[1,4],[3,1], [5,2]])
np.argsort(a, axis=0)

array([[0, 1],
       [1, 2],
       [2, 0]])

### np.argsortのreturnを用いた配列操作

In [13]:
ind = np.argsort(a, axis=0)
np.all(np.take_along_axis(a, ind, axis = 0) == np.sort(a, axis = 0))

True

In [14]:
ind = np.argsort(a, axis=1)
np.all(np.take_along_axis(a, ind, axis = 1) == np.sort(a, axis = 1))

True

## 3. Quicksort

基準値を決めて、それより大きい数と小さい数のグループに分ける。そして、それらのグループに対しても同じことを繰り返していく方法のことを指す。

実際の動きは、

In [15]:
np.random.seed(42)
a = np.random.choice(10, 10, replace=False)
a

array([8, 1, 5, 0, 7, 2, 9, 4, 3, 6])

という配列を例とすると、
1. 値域の平均をとってまず基準値を`5`とする。
2. 前方から基準値より大きい要素を、後方から基準値より小さい要素をスキャンし、見つかったら入れ替えていく。
3. そして、前方からのスキャンと後方からのスキャンがぶつかるまで続ける。
4. 基準値よりも大きいグループと小さいグループに配列をに分割する。
5. それぞれの配列に対して、`(1)~(4)`を繰り返し、全てのグループで入れ替え（配列サイズが2になるまで`(1)~(4)`を繰り返す）が済んだら終了

<img src = 'https://github.com/RyoNakagami/omorikaizuka/blob/master/algorithm/quicksort_image_1.jpg?raw=true'>

### Implementation

In [16]:
def my_quicksort(array, pivot = None):
    """
    INPUT
        array: 1_D array-like
        pivot: sort pivot method
    RETURN
        sorted array
    """
    if len(array) < 2:
        return array
    
    if pivot == 'mean':
        pivot = sum(array)/len(array)
    else:
        pivot = (array[0] + array[-1])/2
    
    left, right = 0, len(array) - 1
    while(left < right):
        while(array[left] < pivot):
            left += 1
        
        while(array[right] > pivot):
            right -= 1
        if (left < right):
            array[left], array[right] = array[right], array[left]
        else:
            break
    
    array[:left] = my_quicksort(array[:left])
    array[left+1::] = my_quicksort(array[left+1:]) 
    
    return array


In [17]:
my_quicksort(a)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

## Appendix: fieldとは？

- a structured data typeにおいて、各sub-typeのことを`field`と呼ぶ。
- `field`は`name`と`type`を持つ

In [18]:
dt = np.dtype([('name', np.unicode_, 16), 
               ('grades', np.float64, (2,))])
x = np.array([('Sarah', (8.0, 7.0)), ('John', (6.0, 7.0))], dtype=dt)

In [19]:
x['name']

array(['Sarah', 'John'], dtype='<U16')

In [20]:
x['grades']

array([[8., 7.],
       [6., 7.]])

In [21]:
x[0]

('Sarah', [8., 7.])

In [22]:
x[1]

('John', [6., 7.])